In [36]:
  '''
  PUSING MY COLLAB PROJECT TO GITHUB
  '''
# !ssh-keygen -t rsa -b 4096 -C "fabianbernard49@gmail.com" -f ~/.ssh/id_rsa -N ""  #Create ssh pair key
# !cat ~/.ssh/id_rsa.pub #Print out your
!ssh-keyscan -H github.com >> ~/.ssh/known_hosts #Make github a knownhost
!ssh -T git@github.com #Aunthenticate github

  # Verify your identity
!git config --global user.name "coded-by-49"
!git config --global user.email "fabianbernard49@gmail.com"

# %cd /content/drive/MyDrive/Git_repos #Create a folder for git hub repos if its your first time
# %cd /content/drive/MyDrive/Git_repos/Telecom-churn-model/ #Enter that folder
!git clone git@github.com:coded-by-49/Telecom-churn-model.git  #Clone your repo into it

!cp "/content/drive/MyDrive/Colab Notebooks/Tele_churn.ipynb" /content/drive/MyDrive/Git_repos/Telecom-churn-model/ #Copy your project note book into it
!git status
!git add Tele_churn.ipynb #Add the file itself to avoid json file upload in repo
!git commit -m "Implementation and Testing of Random forest classifier"
!git push origin main

# github.com:22 SSH-2.0-934fdb2c
# github.com:22 SSH-2.0-934fdb2c
# github.com:22 SSH-2.0-934fdb2c
# github.com:22 SSH-2.0-934fdb2c
# github.com:22 SSH-2.0-934fdb2c
Hi coded-by-49! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'Telecom-churn-model'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 5), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 11.83 KiB | 11.83 MiB/s, done.
Resolving deltas: 100% (5/5), done.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [27]:
from itertools import groupby
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

'''Data Preprocessing'''
from google.colab import drive
import urllib

# Importing file from googledrive
drive.mount('/content/drive')
tele_churn_csv = "/content/drive/MyDrive/GOOGLE COLLAB PROJECTS/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df_tele = pd.read_csv(tele_churn_csv)

# Cleaning out empty feature spots
df_tele["TotalCharges"] = df_tele["TotalCharges"].replace(" ", np.nan)
df_tele.dropna(subset = ["TotalCharges"], inplace=True)
df_tele["TotalCharges"] = pd.to_numeric(df_tele["TotalCharges"])

# Dropping unimportant features
df_tele.drop("customerID", axis = 1, inplace = True) # removed this unimportant feature

# Importation of sckitlearns logisticregression algorithm
from sklearn.linear_model import LogisticRegression

# Encoding our dataframe
encoded_df_tele = pd.get_dummies(df_tele, drop_first= True )
encoded_df_tele.head()

# Conversion of boolean columns into zeros and ones
bool_cols = encoded_df_tele.select_dtypes(include = "bool").columns
encoded_df_tele[bool_cols] = encoded_df_tele[bool_cols].astype(int)

# Conversion of tenure datatype numerical datatype
encoded_df_tele["tenure"] = encoded_df_tele["tenure"].astype(float)

# Storing all the numerical feature titles
all_numerical_features = encoded_df_tele.select_dtypes(include = np.number).columns.tolist()
needed_numerical_feature = [feature for feature in all_numerical_features if encoded_df_tele[feature].nunique()>2]

# Separation of future and labels
labels = encoded_df_tele["Churn_Yes"]
features = encoded_df_tele.drop("Churn_Yes", axis = 1)

# Creating our testing and training data
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size = 0.2, random_state = 42 )

# Scaling our non-categoical numerical features
Scaler = StandardScaler()
x_train[needed_numerical_feature] = Scaler.fit_transform(x_train[needed_numerical_feature])
x_test[needed_numerical_feature] = Scaler.transform(x_test[needed_numerical_feature])

'''Training our models'''
log_regression = LogisticRegression(max_iter=1000)
log_regression.fit(x_train, y_train)

# Testing the model for accuracy
sk_predicted_class = log_regression.predict(x_test)


"""IMPLEMENTATION OF LOGISTIC REGRESSION ALGORITHM FROM SCRATCH"""
class custom_Log_regression:
  '''
      Attributes:
        learning_rate (float): Step size for gradient descent updates.
        n_iters (int): Maximum number of iterations for training.
  '''
  def __init__(self, learning_rate = 0.01, n_iters = 1000):
    self.learning_rate = learning_rate
    self.n_iters = n_iters

  def sigmoid(self,z):
    '''
       Sigmoid function, used for transformation of numerical prediction to 1's and 0's
       Args:
            z (numpy.ndarray): Input values (linear combination of features and weights).
       Our sigmoid then transfroms it into an array numpy.ndarray of probablilities between 0 and 1
    '''
    return 1/(1+np.exp(-z))

  def fit(self,X,Y):
    '''
    (Training function) Fit function would derive the weight and bias that
     occurs during the two minimum derivative of the loss function

    Args:
        X (numpy.ndarray): Training features (shape: [n_samples, n_features]).
        Y (numpy.ndarray): Training labels (shape: [n_samples]).
    '''
    self.num_samples, self.num_features = X.shape
    self.weights = np.zeros(self.num_features)
    self.bias = 0

    for _ in range(self.n_iters):
      y_num_prediction = np.dot(X, self.weights) + self.bias
      y_predicted = self.sigmoid(y_num_prediction)

      # Derivative of loss function
      dw = np.dot(X.T,(y_predicted - Y))/self.num_samples
      db = np.mean((y_predicted - Y))

      # Adjustment weights and bias after each iteration
      self.weights = self.weights - self.learning_rate * dw
      self.bias = self.bias - self.learning_rate * db

      # Stopping condition with eculidean normalisation
      if np.linalg.norm(db) < 0.001 and np.linalg.norm(dw) < 0.001:
        break # Minimum has been derived

  def score(self, x_test,y_test):
    '''
    Scoring function which tells us the confusion metrics (Accuracy) of our model
    '''

    # Deducing model prediction
    y_num_prediction = np.dot(x_test, self.weights) + self.bias
    self.y_predicted = self.sigmoid(y_num_prediction)

    # Addition of decision thresold to get standard decsion (1 or 0)
    self.y_predicted = np.where(self.y_predicted > 0.5, 1, 0)
    accuracy =  np.mean(self.y_predicted == y_test)
    return accuracy

  def confusion_matrix(self,predicted_class,actual_class):
    '''
      The Accuracy/scoring of our model is not enough for solving the business problem

      Confusion metrics would there procuce more insights like:
      Sensitivity and Precision
    '''
    confusion_matrix = {
    'TP': 0,
    'TN': 0,
    'FP': 0,
    'FN': 0
    }
    for i in range(len(actual_class)):
      if predicted_class[i] == actual_class.iloc[i]:
        if predicted_class[i] == 1:
            confusion_matrix["TP"] += 1
        else:
            confusion_matrix["TN"] += 1
      else:
        if actual_class.iloc[i] == 0:
            confusion_matrix["FP"] += 1
        else:
            confusion_matrix["FN"] += 1
    return confusion_matrix


'''Testing our model against the sckit learn version'''
custom_model = custom_Log_regression()
custom_model.fit(x_train, y_train)

print(f"Testing accuracy for sckit learns model: {log_regression.score(x_test, y_test)}")
print(f"Testing accuracy for custom_log_regression : {custom_model.score(x_test,y_test)}")

'''
Scikit-learn Logistic Regression: 79% accuracy.
Custom Logistic Regression: 79/80% accuracY.
'''

'''Testing the confusion matrix'''
# Accessing the predicted value of y from within my custom model
predicted_class = custom_model.y_predicted

# Accessing the actual value of y from within my custom model
actual_class = y_test

print(f"Confusion matrix of sk learn model = {custom_model.confusion_matrix(sk_predicted_class,actual_class)}")
print(f"Confusion matrix of custom_log_regression = {custom_model.confusion_matrix(predicted_class,actual_class)}\n\n")

''' So from our confusion matrix test, we can tell that our data is skwed, because of the poor recall/senstivity (a whooping 45%). '''

''' Implementation of Adasyin-SMOTE
    We want to find most skwed points of class and generate sample features and labels there
    However this algorithm is going to be fitted only to our training data as the main aim is to builds to models prediction.
'''

# Importing algorithm
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(random_state=42)

# Resampling of Futures and Lables
x_train_bal, y_train_bal = adasyn.fit_resample(x_train,y_train)

# Fitting our model with the new Dataset
custom_model.fit(x_train_bal,y_train_bal)
print(f"Accuracy on oversampled dataset prediction by custom_log_regression) = {custom_model.score(x_test,y_test)}")

predicted_class = custom_model.y_predicted
actual_class = y_test

print(f"Confusion matrix based on oversampled dataset prediction by custom_log_regression= {custom_model.confusion_matrix(predicted_class,actual_class)}\n\n")

'''
Testing result of oversampled dataset (ADASYIN SMOTE) traning with custom_log_regression
Custom Logistic Regression: 73% accuracy(balanced) with 76.7% (recall).

Which means logistic regression would not cut it for business standard
'''

"""IMPLEMENATION OF RANDOM FOREST CLASSIFIER"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
# train model
sk_rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
sk_rf_model.fit(x_train_bal,y_train_bal)

#access predicted value of y
y_pred = sk_rf_model.predict(x_test)

# test our model
print(f"Accuracy on oversampled dataset prediction by RandomForestClassifier = {sk_rf_model.score(x_test,y_test)}")

print(f"confusion matrix based on oversammpled dataset prediction by RandomForestClassifier  = {custom_model.confusion_matrix(y_pred,y_test)}")

'''
Testing result of oversampled dataset (ADASYIN SMOTE) training with RandomForestClassifier
RandomForestClassifier: 76% accuracy(balanced) with 64.7% (recall).

Which means RandomForestClassifier would not cut it for business standard
'''


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Testing accuracy for sckit learns model: 0.7867803837953091
Testing accuracy for custom_log_regression : 0.7903340440653873
Confusion matrix of sk learn model = {'TP': 192, 'TN': 915, 'FP': 118, 'FN': 182}
Confusion matrix of custom_log_regression = {'TP': 171, 'TN': 941, 'FP': 92, 'FN': 203}


Accuracy on oversampled dataset prediction by custom_log_regression) = 0.7356076759061834
Confusion matrix based on oversampled dataset prediction by custom_log_regression= {'TP': 287, 'TN': 748, 'FP': 285, 'FN': 87}


Accuracy on oversampled dataset prediction by RandomForestClassifier = 0.7604832977967306
confusion matrix based on oversammpled dataset prediction by RandomForestClassifier  = {'TP': 242, 'TN': 828, 'FP': 205, 'FN': 132}


'\nTesting result of oversampled dataset (ADASYIN SMOTE) set traning with RandomForestClassifier\nRandomForestClassifier: 76% accuracy(balanced) with 64.7% (recall).\n\nWhich means RandomForestClassifier would not cut it for business standard\n'